# Recommendation 기본 알고리즘

## 2.1. Popularity, High Rated Based(가장 단순함)
- 가장 쉽게 인기도, 즉 높은 평점을 갖는 item을 추천 가능
- 모두에게 동일한 item이 추천됨

In [1]:
ratings = {
    'Dave': {'달콤한인생': 5, '범죄도시': 3, '샤인': 3},
    'David': {'달콤한인생': 5, '범죄도시': 1, '샤인': 4},
    'Alex': {'달콤한인생': 0, '범죄도시': 4, '샤인': 5},
    'Andy': {'달콤한인생': 2, '범죄도시': 1, '샤인': 5}
}

- 영화 중 가장 평점이 높은 순으로 추천해주면 됨

In [14]:
# 평점이 가장 높은 두 영화 출력하기
movie_dict = dict()
for rating in ratings:
    for movie in ratings[rating].keys():
        if movie not in movie_dict:
            movie_dict[movie] = ratings[rating][movie]
        else:
            movie_dict[movie] += ratings[rating][movie]

for movie in ratings[rating].keys():
    movie_dict[movie] = movie_dict[movie] / len(ratings)

print(sorted(movie_dict))

import operator
sorted_x = sorted(movie_dict.items(), key=operator.itemgetter(1), reverse=True)

print(sorted_x)

['달콤한인생', '범죄도시', '샤인']
[('샤인', 4.25), ('달콤한인생', 3.0), ('범죄도시', 2.25)]


## 2.2. Collaborative Filtering

- 데이터 구성 : 사용자가 입력한 선호도(평점)를 사용하여 사용자-항목 선호도(평점) **행렬**을 만든다.
- 유사도 계산 : 1 단계에서 만들어진 행렬을 사용하여 사용자들 간의 유사도를 계산한다.
- 예측 값 계산 및 추천 목록 생성 : 사용자들 간의 유사도를 바탕으로 모든 항목에 대해 예측 값을 계산하고 높은 예측 값을 갖는 상위 N개의 추천 목록을 생성한다.

### 1. 데이터 구성

In [15]:
ratings = {
    'Dave':{'달콤한인생':5,'범죄도시':3,'샤인':3},
    'David':{'달콤한인생':2,'범죄도시':1,'샤인':4},
    'Alex':{'범죄도시':4,'샤인':5},
    'Andy':{'달콤한인생':2,'범죄도시':1,'샤인':5}
}

In [16]:
ratings.get('Dave')

{'달콤한인생': 5, '범죄도시': 3, '샤인': 3}

In [17]:
ratings.get('Dave').get('샤인')

3

### 2. 유사도 계산

In [18]:
import math
def sim(i, j):
    return math.sqrt(pow(i, 2) + pow(j, 2))

In [20]:
var1 = ratings['Alex']['범죄도시']\
    - ratings['Andy']['범죄도시'] # 4 - 1

In [21]:
var1

3

In [22]:
var2 = ratings['Alex']['샤인'] - ratings['Andy']['샤인'] # 5 - 5

In [23]:
var2

0

In [24]:
sim(var1, var2)

3.0

#### Alex가 평가한 범죄도시, 샤인을 모두 평가한 사용자와 모두 거리 구하기(유사도)

In [26]:
for rating in ratings:
    if rating != 'Alex':
        rat1 = ratings.get('Alex').get('범죄도시') - ratings.get(rating).get('범죄도시')
        rat2 = ratings.get('Alex').get('샤인') - ratings.get(rating).get('샤인')
        print(rating, " : ", sim(rat1, rat2))

Dave  :  2.23606797749979
David  :  3.1622776601683795
Andy  :  3.0


#### 정규화를 통해 유사도 범위를 0과 1사이로 가두고, 1에 가까울 수록 유사도가 높게 변경하기

In [27]:
for rating in ratings:
    if rating != 'Alex':
        rat1 = ratings.get('Alex').get('범죄도시') - ratings.get(rating).get('범죄도시')
        rat2 = ratings.get('Alex').get('샤인') - ratings.get(rating).get('샤인')
        print(rating, " : ", 1 / (1 + sim(rat1, rat2)))

Dave  :  0.3090169943749474
David  :  0.2402530733520421
Andy  :  0.25


#### Dave가 평가한 범죄도시와 샤인 모두 평가한 사용자와의 거리를 구해서, 가장 Dave와 유사한 사용자 구하기

In [30]:
for rating in ratings:
    if rating != 'Dave':
        rat1 = ratings.get('Dave').get('범죄도시') - ratings.get(rating).get('범죄도시')
        rat2 = ratings['Dave']['샤인'] - ratings[rating]['샤인']
        print(rating, " : ", 1 / (1 + sim(rat1, rat2)))

David  :  0.3090169943749474
Alex  :  0.3090169943749474
Andy  :  0.2612038749637414


## 3. 유사도(Similarity) 계산 방식
- 3.1. 평균제곱차이 유사도(Mean Squared Difference Similarity)
- 3.2. 코사인 유사도(Cosine Similarity)
- 3.3. 피어슨 유사도(Pearson Similarity)

### 3.1. 평균제곱차이 유사도(Mean Squared Difference Similarity)

In [31]:
def sim_msd(data, name1, name2):
    sum = 0
    count = 0
    for movies in data[name1]:
        if movies in data[name2]:
            sum += pow(data[name1][movies] - data[name2][movies], 2)
            count += 1
    return 1 / (1 + (sum / count))

In [32]:
sim_msd(ratings, 'Dave', 'Alex')

0.2857142857142857

In [33]:
sim_msd(ratings, 'Dave', 'Andy')

0.15

In [43]:
for user in ratings:
    if user != 'Dave':
        print("Dave와 {0}의 MSD Similarity : {1}"\
              .format(user, sim_msd(ratings, 'Dave', user)))

Dave와 David의 MSD Similarity : 0.1764705882352941
Dave와 Alex의 MSD Similarity : 0.2857142857142857
Dave와 Andy의 MSD Similarity : 0.15


### 3.2. 코사인 유사도 (Cosine Similarity)

In [34]:
import math
def sim_cosine(data, name1, name2):
    sum_name1 = 0
    sum_name2 = 0
    sum_all = 0
    count = 0
    
    for movies in data[name1]:
        if movies in data[name2]:
            sum_name1 += pow(data[name1][movies], 2)
            sum_name2 += pow(data[name2][movies], 2)
            sum_all += data[name1][movies] * data[name2][movies]
        
    return sum_all / (math.sqrt(sum_name1) * math.sqrt(sum_name2))

In [38]:
for user in ratings:
    if user != 'Dave':
        print("Dave와 {0}의 Cosine Similarity : {1}"\
              .format(user, sim_cosine(ratings, 'Dave', user)))

Dave와 David의 Cosine Similarity : 0.8319479070496523
Dave와 Alex의 Cosine Similarity : 0.9938837346736189
Dave와 Andy의 Cosine Similarity : 0.7795844649455863


### 3.3. 피어슨 유사도 (Pearson Similarity)

In [41]:
def sim_pearson(data, name1, name2):
    avg_name1 = 0
    avg_name2 = 0
    count = 0
    for movies in data[name1]:
        if movies in data[name2]: #같은 영화를 봤다면
            avg_name1 = data[name1][movies]
            avg_name2 = data[name2][movies]
            count += 1
    
    avg_name1 = avg_name1 / count
    avg_name2 = avg_name2 / count
    
    sum_name1 = 0
    sum_name2 = 0
    sum_name1_name2 = 0
    count = 0
    for movies in data[name1]:
        if movies in data[name2]: #같은 영화를 봤다면
            sum_name1 += pow(data[name1][movies] - avg_name1, 2)
            sum_name2 += pow(data[name2][movies] - avg_name2, 2)
            sum_name1_name2 += (data[name1][movies] - avg_name1) * (data[name2][movies] - avg_name2)
    
    return sum_name1_name2 / (math.sqrt(sum_name1)*math.sqrt(sum_name2))

In [42]:
for user in ratings:
    if user != 'Dave':
        print("Dave와 {0}의 Pearson Similarity : {1}"\
              .format(user, sim_pearson(ratings, 'Dave', user)))

Dave와 David의 Pearson Similarity : 0.5406205059012895
Dave와 Alex의 Pearson Similarity : 0.970142500145332
Dave와 Andy의 Pearson Similarity : 0.39840953644479793


In [44]:
def top_match(data, name, index=3, sim_function=sim_pearson):
    list1 = []
    for user in data:
        if name != user:
            list1.append((sim_function(data, name, user), user))
    list1.sort()
    list1.reverse()
    return list1[:index]

In [45]:
top_match(ratings, 'Dave')

[(0.970142500145332, 'Alex'),
 (0.5406205059012895, 'David'),
 (0.39840953644479793, 'Andy')]

In [47]:
top_match(ratings, 'Alex', sim_function=sim_msd)

[(0.2857142857142857, 'Dave'),
 (0.18181818181818182, 'Andy'),
 (0.16666666666666666, 'David')]